In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from BigQueryDataFetcher import BigQueryDataFetcher
import ActiveStrategyFramework as asf
from SimpleResetStrategy import SimpleResetStrategy
import UNI_v3_funcs

import config
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = config.GOOGLE_SERVICE_AUTH_JSON

fetcher = BigQueryDataFetcher()  # Add project_id or credentials_path if needed

# UNI-WETH 0.3% pool
uni_weth_address = '0x1d42064fc4beb5f8aaf85f4617ae8b3b5b8bd801'
date_begin = '2025-06-01'
date_end = '2025-06-30'
swap_data = fetcher.fetch_pool_data(
    uni_weth_address, date_begin, date_end, decimals_0=18, decimals_1=18,
    network='mainnet', save_format='csv', overwrite=True
)

# UNI-USDC 0.3% pool for USD prices (token_0_usd_data)
uni_usdc_address = '0xd0fc8ba7e267f2bc56044a7715a489d851dc6d78'
uni_usdc_data = fetcher.fetch_pool_data(
    uni_usdc_address, date_begin, date_end, decimals_0=18, decimals_1=6,
    network='mainnet', save_format='csv', overwrite=True
)

# Aggregate price data (1-min resolution)
price_data = swap_data['quotePrice'].resample('1T').last().ffill()

# Token0 USD data (UNI/USD = USDC/UNI from UNI-USDC pool)
token_0_usd_data = uni_usdc_data['quotePrice'].resample('1T').last().ffill().to_frame('quotePrice')

# Model data: Daily returns for ECDF (using full fetched data)
model_data = pd.DataFrame({'quotePrice': price_data}).resample('D').last()
model_data['price_return'] = model_data['quotePrice'].pct_change().dropna()

c:\users\z640\appdata\local\programs\python\python39\lib\site-packages\google\cloud\bigquery\table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


TypeError: unsupported operand type(s) for /: 'str' and 'float'